In [15]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
import torch as tr 
import numpy as np 
import pandas as pd 
from collections import defaultdict
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from src.lda_model import LDASmoothed 
from src.utils import get_vocab_from_docs, get_np_wct, data_loader 
from pprint import pprint 
import copy 

In [37]:
docs_raw_dict, raw_word_2_idx, raw_idx_2_word = data_loader('ap')


There are 2250 documents in the dataset
On average estimated document length is 406.66133333333335 words per document
There are 10473 unique vocab in the Associate Press Dataset


In [38]:
word_2_idx 

{'i': 0,
 'new': 1,
 'percent': 2,
 'people': 3,
 'year': 4,
 'two': 5,
 'million': 6,
 'president': 7,
 'last': 8,
 'government': 9,
 'years': 10,
 'first': 11,
 'police': 12,
 'state': 13,
 'states': 14,
 'officials': 15,
 'soviet': 16,
 'united': 17,
 'bush': 18,
 'time': 19,
 'three': 20,
 'billion': 21,
 'today': 22,
 'national': 23,
 'told': 24,
 'american': 25,
 'thursday': 26,
 'federal': 27,
 'house': 28,
 'week': 29,
 'court': 30,
 'day': 31,
 'tuesday': 32,
 'made': 33,
 'news': 34,
 'wednesday': 35,
 'monday': 36,
 'friday': 37,
 'say': 38,
 'company': 39,
 'city': 40,
 'party': 41,
 'just': 42,
 'group': 43,
 'york': 44,
 'market': 45,
 'report': 46,
 'department': 47,
 'military': 48,
 'south': 49,
 'union': 50,
 'members': 51,
 'home': 52,
 'west': 53,
 'political': 54,
 'reported': 55,
 'make': 56,
 'going': 57,
 'office': 58,
 'get': 59,
 'spokesman': 60,
 'dont': 61,
 'world': 62,
 'like': 63,
 'four': 64,
 'think': 65,
 'committee': 66,
 'back': 67,
 'work': 68,
 'de

In [150]:
wct_dict = get_vocab_from_docs(docs)
wct_dict 

{'My': [1, 0, 0, 0],
 'name': [1, 0, 0, 0],
 'is': [1, 1, 0, 0],
 'tuo': [2, 2, 1, 1],
 '.': [1, 0, 0, 0],
 'cute': [0, 1, 0, 0],
 '!': [0, 1, 1, 1],
 'Tuo': [0, 0, 1, 0],
 'likes': [0, 0, 1, 0],
 'to': [0, 0, 1, 0],
 'poo': [0, 0, 1, 0],
 'a': [0, 0, 1, 0],
 'lot': [0, 0, 1, 0],
 'mama': [0, 0, 0, 1],
 'bear': [0, 0, 0, 1],
 'very': [0, 0, 0, 1]}

In [151]:
wct_array, w_2_idx = get_np_wct(wct_dict, docs)

print(wct_array)
pprint(w_2_idx)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [2. 2. 1. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
{'!': 6,
 '.': 4,
 'My': 0,
 'Tuo': 7,
 'a': 11,
 'bear': 14,
 'cute': 5,
 'is': 2,
 'likes': 8,
 'lot': 12,
 'mama': 13,
 'name': 1,
 'poo': 10,
 'to': 9,
 'tuo': 3,
 'very': 15}


In [152]:
wct_array.astype(object)

array([[1.0, 0.0, 0.0, 0.0],
       [1.0, 0.0, 0.0, 0.0],
       [1.0, 1.0, 0.0, 0.0],
       [2.0, 2.0, 1.0, 1.0],
       [1.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0.0],
       [0.0, 1.0, 1.0, 1.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 1.0, 0.0],
       [0.0, 0.0, 0.0, 1.0],
       [0.0, 0.0, 0.0, 1.0],
       [0.0, 0.0, 0.0, 1.0]], dtype=object)

In [155]:
docs_in_vocab_indx = copy.deepcopy(docs)
#print(docs_in_vocab_indx)

for id, d in enumerate(docs): 

    for n in range(len(d)): 

        docs_in_vocab_indx[id][n] = w_2_idx[docs[id][n]]

print(docs_in_vocab_indx)
print(docs)
# docs_in_idx = np.array(docs_in_vocab_indx)
# docs_in_idx

[[0, 1, 2, 3, 3, 4], [3, 3, 2, 5, 6], [7, 3, 8, 9, 10, 11, 12, 6], [13, 14, 15, 3, 6]]
[['My', 'name', 'is', 'tuo', 'tuo', '.'], ['tuo', 'tuo', 'is', 'cute', '!'], ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'], ['mama', 'bear', 'very', 'tuo', '!']]


- construct  new phi 

In [161]:
docs  

[['My', 'name', 'is', 'tuo', 'tuo', '.'],
 ['tuo', 'tuo', 'is', 'cute', '!'],
 ['Tuo', 'tuo', 'likes', 'to', 'poo', 'a', 'lot', '!'],
 ['mama', 'bear', 'very', 'tuo', '!']]

In [157]:
phi = tr.zeros(len(docs),wct_array.shape[0],num_topics)

for id, d in enumerate(docs): 

    for word in d: 

        v = w_2_idx[word]

        phi[id][v] = 1/num_topics 

phi 

tensor([[[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 

In [261]:
lda = LDASmoothed(
    docs,
    num_topics, 
    wct_dict, 
    word_ct_array = wct_array
)
#LDASmoothed(docs, 3, wct_dict)

Word mapping to vocab index is: 
{'!': 6,
 '.': 4,
 'My': 0,
 'Tuo': 7,
 'a': 11,
 'bear': 14,
 'cute': 5,
 'is': 2,
 'likes': 8,
 'lot': 12,
 'mama': 13,
 'name': 1,
 'poo': 10,
 'to': 9,
 'tuo': 3,
 'very': 15}
Topic Dirichlet Prior, Alpha
torch.Size([3])
tensor([1.0662, 1.0646, 1.0246], dtype=torch.float64)

Word Dirichlet Prior, Eta
torch.Size([16])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)

Var Inf - Word Dirichlet prior, Lambda
torch.Size([3, 16])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       dtype=torch.float64)

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([4, 3])
tensor([[6.3996, 6.3979, 6.3580],
        [6.3996, 6.3979, 6.3580],
        [6.3996, 6.3979, 6.3580],
        [6.3996, 6.3979, 6.3580]], dtype=torch.float64)

Var -Inf - Wor

In [246]:
lda.e_step()

Iteration 0, Delta Gamma = inf, the ELBO is -85.68776167935472
Iteration 1, Delta Gamma = 11.634543358698698, the ELBO is -78.76652059022041
Iteration 2, Delta Gamma = 0.3802251983925746, the ELBO is -78.69498753371093
Iteration 3, Delta Gamma = 0.37955856202224414, the ELBO is -78.61939065204434
Iteration 4, Delta Gamma = 0.3934062072450881, the ELBO is -78.53391581308772
Iteration 5, Delta Gamma = 0.4166941230683475, the ELBO is -78.43448663806691
Iteration 6, Delta Gamma = 0.44447213643639694, the ELBO is -78.31839275417336
Iteration 7, Delta Gamma = 0.47006831218353495, the ELBO is -78.18563628775681
Iteration 8, Delta Gamma = 0.48500553411935377, the ELBO is -78.04086154141861
Iteration 9, Delta Gamma = 0.4812559933817966, the ELBO is -77.89415487962795
Iteration 10, Delta Gamma = 0.45526726021920205, the ELBO is -77.75867198536456
Iteration 11, Delta Gamma = 0.4108527412220289, the ELBO is -77.64531724002988
Iteration 12, Delta Gamma = 0.35787759461683966, the ELBO is -77.5582887

In [247]:
lda._phi_

tensor([[[0.8328, 0.1015, 0.0656],
         [0.8328, 0.1015, 0.0656],
         [0.8890, 0.0676, 0.0434],
         [0.9329, 0.0338, 0.0333],
         [0.8328, 0.1015, 0.0656],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.8823, 0.0674, 0.0503],
         [0.9276, 0.0338, 0.0387],
         [0.0000, 0.0000, 0.0000],
         [0.8211, 0.1014, 0.0775],
         [0.8271, 0.0731, 0.0997],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 

In [248]:
lda.update_alpha()

Iteration 0, Delta Alpha = 0.49748160035002437 elbo is -77.08556890799193
Iteration 1, Delta Alpha = 0.4482049669844789 elbo is -76.38807636805365
Iteration 2, Delta Alpha = 0.13527603583706238 elbo is -76.26336597159539
Iteration 3, Delta Alpha = 0.008514981080721542 elbo is -76.25669591767722
Iteration 4, Delta Alpha = 3.059578285176301e-05 elbo is -76.25667309490211
Iteration 5, Delta Alpha = 3.988947973825057e-10 elbo is -76.256673094607


In [249]:
lda._alpha_

tensor([2.2114, 0.8318, 1.0284], dtype=torch.float64)

In [250]:
lda._eta_

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)

In [251]:
lda.update_eta()

Iteration 0, Delta Alpha = 0.7782684947672688, elbo is -76.256673094607
Iteration 1, Delta Alpha = 0.42408186920084473, elbo is -75.08279437667072
Iteration 2, Delta Alpha = 0.07483985966584267, elbo is -74.91299061680559
Iteration 3, Delta Alpha = 0.0018411311094690805, elbo is -74.90918237262353
Iteration 4, Delta Alpha = 1.115209823462873e-06, elbo is -74.909180145797
Iteration 5, Delta Alpha = 4.317222680992514e-13, elbo is -74.90918014579614


In [252]:
lda._eta_

tensor([1.1367, 1.1367, 1.2994, 1.9299, 1.1367, 1.1389, 1.6381, 1.1391, 1.1391,
        1.1391, 1.1391, 1.1391, 1.1391, 1.1575, 1.1575, 1.1575],
       dtype=torch.float64)

In [219]:
lda._phi_

tensor([[[0.0631, 0.0851, 0.8518],
         [0.0631, 0.0851, 0.8518],
         [0.0427, 0.0588, 0.8985],
         [0.0415, 0.0510, 0.9075],
         [0.0631, 0.0851, 0.8518],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0584, 0.0815, 0.8601],
         [0.0570, 0.0709, 0.8721],
         [0.0000, 0.0000, 0.0000],
         [0.0908, 0.1261, 0.7831],
         [0.1572, 0.2023, 0.6405],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 

In [262]:
lda.fit()

Training started -> ELBO at init is :-85.37606416585183
ELBO converged at 1629 -> ELBO:-60.85687311861693
